# NRC Emotional Lexicon

This is the [NRC Emotional Lexicon](http://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm): "The NRC Emotion Lexicon is a list of English words and their associations with eight basic emotions (anger, fear, anticipation, trust, surprise, sadness, joy, and disgust) and two sentiments (negative and positive). The annotations were manually done by crowdsourcing."

I don't trust it, but everyone uses it.

In [2]:
import pandas as pd

In [8]:
filepath = "./NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
emolex_df = pd.read_csv(filepath,  names=["word", "emotion", "association"], skiprows=45, sep='\t')
emolex_df.head(12)

,word,emotion,association
0,abandonment,joy,0
1,abandonment,negative,1
2,abandonment,positive,0
3,abandonment,sadness,1
4,abandonment,surprise,1
5,abandonment,trust,0
6,abate,anger,0
7,abate,anticipation,0
8,abate,disgust,0
9,abate,fear,0


Seems kind of simple. A column for a word, a column for an emotion, and whether it't associated or not. You see "aback aback aback aback" because there's a row for every word-emotion pair.

## What emotions are covered?

Let's look at the 'emotion' column. What can we talk about?

In [9]:
emolex_df.emotion.unique()

array(['joy', 'negative', 'positive', 'sadness', 'surprise', 'trust',
       'anger', 'anticipation', 'disgust', 'fear'], dtype=object)

In [10]:
emolex_df.emotion.value_counts()

sadness         14178
trust           14178
negative        14178
positive        14178
joy             14178
surprise        14178
fear            14177
anger           14177
disgust         14177
anticipation    14177
Name: emotion, dtype: int64

## How many words does each emotion have?

Each emotion doesn't have 14182 words associated with it, unfortunately! `1` means "is associated" and `0` means "is not associated."

We're only going to care about "is associated."

In [11]:
emolex_df[emolex_df.association == 1].emotion.value_counts()

negative        3322
positive        2312
fear            1473
anger           1245
trust           1230
sadness         1189
disgust         1058
anticipation     839
joy              689
surprise         534
Name: emotion, dtype: int64

In theory things could be *kind of* angry or *kind of* joyous, but it doesn't work like that. If you want to spend a few hundred dollars on Mechnical Turk, though, *your own personal version can.*

## What if I just want the angry words?

In [12]:
emolex_df[(emolex_df.association == 1) & (emolex_df.emotion == 'anger')].word

126             abhor
136         abhorrent
226           abolish
256       abomination
586             abuse
             ...     
141176       wrongful
141186        wrongly
141426           yell
141456           yelp
141596          youth
Name: word, Length: 1245, dtype: object

## Reshaping

You can also reshape the data in order to look at it a slightly different way

In [13]:
emolex_words = emolex_df.pivot(index='word', columns='emotion', values='association').reset_index()
emolex_words.head()

emotion,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,abandonment,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,1.0,0.0
2,abate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,abatement,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,abba,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


You can now pull out individual words...

In [14]:
# If you didn't reset_index you could do this more easily
# by doing emolex_words.loc['charitable']
emolex_words[emolex_words.word == 'charitable']

emotion,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
1998,charitable,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


...or individual emotions....

In [15]:
emolex_words[emolex_words.anger == 1].head()

emotion,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
14,abhor,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
15,abhorrent,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
24,abolish,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
27,abomination,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
60,abuse,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0


...or multiple emotions!

In [16]:
emolex_words[(emolex_words.joy == 1) & (emolex_words.negative == 1)].head()

emotion,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
58,abundance,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
1015,balm,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
1379,boisterous,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
1913,celebrity,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
2001,charmed,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0


The useful part is going to be just getting words for a **single emotion.**

In [17]:
# Angry words
emolex_words[emolex_words.anger == 1].word

14             abhor
15         abhorrent
24           abolish
27       abomination
60             abuse
            ...     
14118       wrongful
14119        wrongly
14143           yell
14146           yelp
14160          youth
Name: word, Length: 1245, dtype: object